In [1]:
import numpy as np
import re
from graphviz import Digraph
import json

In [2]:
def getOpenParent(Locations, value):
    loc = Locations[0]
    for i in range(len(Locations)):
        if Locations[i] > value:
            break
        else:
            loc = Locations[i]
    return loc


# getOpenParent([0,10,11], 6)

In [37]:
# inp = "((a|(bc*|s)|d)*e)|(i)"
def solve(ChList):
    NodeCount = 0
    NFA = []
    while len(ChList) != 1:
        index_open = np.where(ChList == '(')[0]
        index_closed = np.where(ChList == ')')[0]

        loc_C = index_closed[0]
        loc_O = getOpenParent(index_open, loc_C)

        reg_list = np.concatenate((ChList[loc_O+1 :loc_C], np.array(['|'])))    
#         print(reg_list)

        idx_list = [idx + 1 for idx, val in enumerate(reg_list) if val == '|']
        lists_split_Or = [reg_list[i: j-1] for i, j in
                zip([0] + idx_list, idx_list + ([len(reg_list)] if idx_list[-1] != len(reg_list) else []))]

#         print(lists_split_Or)

        subNFA_to_or = []
        for subList in lists_split_Or:
            subNFA = []
            for j in range(len(subList)):
                print(subList[j])
                if subList[j] == '*':
                    NFA.append([])
                    NFA[subNFA[len(subNFA)-1][0]].append((NodeCount, 'epsilon'))
                    NFA[subNFA[len(subNFA)-1][1]].append((subNFA[len(subNFA)-1][0], 'epsilon'))
                    NFA[subNFA[len(subNFA)-1][1]].append((NodeCount, 'epsilon'))
                    subNFA[len(subNFA)-1] = (subNFA[len(subNFA)-1][0], NodeCount)
                    NodeCount += 1

                elif type(subList[j]) == tuple:
                    subNFA.append(subList[j])

                else:
                    NFA.append([])
                    NFA.append([])
                    NFA[NodeCount].append((NodeCount+1, subList[j]))
                    subNFA.append((NodeCount, NodeCount+1))
                    NodeCount += 2

#             print(subNFA)

            #to handle anding each sub graph
            for i in range(1, len(subNFA)):
                NFA[subNFA[i-1][1]].append((subNFA[i][0], "epsilon"))

            subNFA_to_or.append((subNFA[0][0], subNFA[len(subNFA)-1][1]))

        print(subNFA_to_or)
        #to handle oring each sub graph
        if len(subNFA_to_or) > 1:
          NFA.append([])
          NFA.append([])
          curNode = NodeCount
          NodeCount += 2
          for tup in subNFA_to_or:
              NFA[curNode].append((tup[0], "epsilon"))
              NFA[tup[1]].append((curNode+1, "epsilon"))
              ChList[loc_O] = (curNode, curNode+1)
                 
        else:
          ChList[loc_O] = subNFA_to_or[0]
        ChList = np.delete(ChList, [i for i in range(loc_O+1,loc_C+1)])
        print(ChList)      

      

    print(NFA)
    return NFA, ChList[0][0], ChList[0][1]

In [38]:
def draw(res,start,end):
        json_file={}
       
        #draw the graph 
        f = Digraph('NFA Graph',format="png")
        f.attr(rankdir='LR', size='8,5')
        
        f.attr('node', shape='doublecircle')
        f.node('S'+str(end)) 
        f.attr('node', shape='circle')
        json_file["startingState"]="S"+str(start)
        f.node('S'+str(start),color="blue")
        #f.node('S'+str(start),color="red")
        state_list=list(range(len(res)))
        state_list.remove(start)
        state_list.insert(0,start)
        for node_inx in state_list:
            terminate=False
            if node_inx==end:
                terminate=True
            curr_state="S"+str(node_inx)
            json_file[curr_state]={
                "isTerminatingState":terminate}
            for edge in res[node_inx]:      
                out_node=str(edge[0])
                inp_char=edge[1]
                if inp_char in json_file[curr_state]:
                     json_file[curr_state][inp_char].append("S"+out_node)
                else:
                    json_file[curr_state][inp_char] = ["S"+out_node]
                
                #draw
                f.edge('S'+str(node_inx), 'S'+str(out_node), label=inp_char)
            
            json_object=json.dumps(json_file, indent=4)
            # Writing to sample.json
            with open("nfa.json", "w") as outfile:
                outfile.write(json_object)
        
        f.view()
        return json_object

In [39]:
def main():
    reg_exp = input("enter the regular expression:   ")
    try:
        re.compile(reg_exp)
    except re.error:
        print("Non valid regex pattern")
        exit()
        
    inp = '('+reg_exp+')'
    ChList = np.array([char for char in inp],dtype=object)  
    
    NFA, start, end = solve(ChList)
    #output json file and draw the graph
   
   
    json_object= draw(NFA,start,end)
    print("\n\n",json_object)

In [40]:
main()

enter the regular expression:   01|23
0
1
2
3
[(0, 3), (4, 7)]
[(8, 9)]
[[(1, '0')], [(2, 'epsilon')], [(3, '1')], [(9, 'epsilon')], [(5, '2')], [(6, 'epsilon')], [(7, '3')], [(9, 'epsilon')], [(0, 'epsilon'), (4, 'epsilon')], []]


 {
    "startingState": "S8",
    "S8": {
        "isTerminatingState": false,
        "epsilon": [
            "S0",
            "S4"
        ]
    },
    "S0": {
        "isTerminatingState": false,
        "0": [
            "S1"
        ]
    },
    "S1": {
        "isTerminatingState": false,
        "epsilon": [
            "S2"
        ]
    },
    "S2": {
        "isTerminatingState": false,
        "1": [
            "S3"
        ]
    },
    "S3": {
        "isTerminatingState": false,
        "epsilon": [
            "S9"
        ]
    },
    "S4": {
        "isTerminatingState": false,
        "2": [
            "S5"
        ]
    },
    "S5": {
        "isTerminatingState": false,
        "epsilon": [
            "S6"
        ]
    },
    "S6":